In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
%load_ext blackcellmagic

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd

In [ ]:
import extract
import preprocess as prep
import feature_engineering as fe
import imputations as imp
import evaluate as eval_

In [ ]:
import sys
sys.path.insert(0,'/home/shared/utils')

## Training

In [ ]:
loans = extract.fetch_funded_mature_loans("'2018-01-01'", "'2020-04-19'")

In [ ]:
loans = prep.preprocess_iloans(loans)

In [ ]:
loans = fe.fe_iloans(loans)

In [ ]:
access_count = extract.get_esign_time_diff("'2018-01-01'", "'2020-04-19'")

In [ ]:
access_count = prep.preprocess_esign(access_count)

In [ ]:
bank_reports = extract.fetch_bank_reports("'2018-01-01'", "'2020-04-19'")

In [ ]:
bank_reports = prep.preprocess_bank_reports(bank_reports)

In [ ]:
bank_reports = fe.fe_bank_reports(bank_reports)

In [ ]:
bank_app = extract.fetch_bank_app_loans("'2018-01-01'", "'2020-04-19'")

In [ ]:
bank_app = prep.preprocess_bank_app(bank_app)

In [ ]:
bank_reports_cols = ['loan_id','LenderAmountDeb', 'LenderCountCred', 'LenderAmountCred30', 'LenderCountDeb',
       'LenderAmountDeb30', 'LenderCountCred30', 'LenderCountDeb30',
       'LenderAmountCred', 'UniqLenderCount']

In [ ]:
learning = pd.merge(loans, access_count, how = 'left',on = 'loan_id')

In [ ]:
learning = pd.merge(learning, bank_reports, how = 'left',on = 'loan_id')

In [ ]:
learning = pd.merge(learning, bank_app, how = 'left', on = 'loan_id')

In [ ]:
BV_status_list = ['Bank Validation Uncertain', 'Bank Validation Approved']

In [ ]:
learning = learning[learning['bank_app_decision'].isin(BV_status_list)]

In [ ]:
learning = learning.loc[learning['primary_account'].notnull(), :]

In [ ]:
training = learning[(learning['OriginationDate'] >= '2018-01-01') & (learning['OriginationDate'] <= '2019-12-31')]

In [ ]:
evaluation = learning[(learning['OriginationDate'] >= '2020-01-01') & (learning['OriginationDate'] <= '2020-04-19')]

In [ ]:
training, imp_acc_count, imp_dti, imp_pay_day = imp.impute_learning(training)

In [ ]:
evaluation[['AccessCount']] = imp_acc_count.transform(evaluation[['AccessCount']])

In [ ]:
evaluation[['dti']] = imp_dti.transform(evaluation[['dti']])

In [ ]:
evaluation[['pay_day_test_result_amount']] = imp_pay_day.transform(evaluation[['pay_day_test_result_amount']])

In [ ]:
evaluation['LeadProvider'] = evaluation['LeadProvider'].fillna('freedom')

In [ ]:
df_loans = training[['MonthlyGrossIncome', 'Age', 'Reloan', 'LeadProvider', 'LenderCountCred30', 
                     'UniqLenderCount', 'LenderAmountDeb', 'LenderAmountCred', 'LenderAmountDeb30',
                     'LenderAmountCred30', 'LenderCountDeb', 'LenderCountCred', 'LenderCountDeb30',
                     'median_weekly_credit', 'dev_weekly_credit_count', 'median_daily_debit', 'AccessCount', 
                     'IsFirstDefault', 'dti', 'pay_day_test_result_amount', 'diff_pos_neg_days', 'median_daily_balance']]

In [ ]:
df_loans = prep.changing_bool_dtypes_to_str(df_loans)

## Pycaret

In [ ]:
from pycaret.classification import *

In [ ]:
clf = setup(data = df_loans, train_size = .99, target = 'IsFirstDefault', session_id = 69,
            normalize = True,
            transformation = True,
            remove_outliers = True, 
            numeric_features = ['MonthlyGrossIncome', 'Age', 'LenderCountCred30', 
                                'UniqLenderCount', 'LenderAmountDeb', 'LenderAmountCred', 'LenderAmountDeb30',
                                'LenderAmountCred30', 'LenderCountDeb', 'LenderCountCred', 'LenderCountDeb30',
                                'median_weekly_credit', 'dev_weekly_credit_count', 'median_daily_debit',
                                'AccessCount', 'dti', 'pay_day_test_result_amount', 'diff_pos_neg_days', 'median_daily_balance'])

In [ ]:
tuned_lr = tune_model('lr', optimize = 'F1')

In [ ]:
bagged_lr = ensemble_model(tuned_lr, method = 'Bagging')

In [ ]:
final_model = finalize_model(bagged_lr)

In [ ]:
save_model(final_model, 'new_model')

## Evaluation

In [ ]:
df_eval = evaluation[['MonthlyGrossIncome', 'Age', 'Reloan', 'LeadProvider', 'LenderCountCred30', 
                     'UniqLenderCount', 'LenderAmountDeb', 'LenderAmountCred', 'LenderAmountDeb30',
                     'LenderAmountCred30', 'LenderCountDeb', 'LenderCountCred', 'LenderCountDeb30',
                     'median_weekly_credit', 'dev_weekly_credit_count', 'median_daily_debit', 'AccessCount', 
                     'IsFirstDefault', 'dti', 'pay_day_test_result_amount', 'diff_pos_neg_days', 'median_daily_balance']]

In [ ]:
df_eval = prep.changing_bool_dtypes_to_str(df_eval)

In [ ]:
new_model = load_model('new_model')

In [ ]:
predictions_pycaret = predict_model(new_model, data = df_eval)

In [ ]:
eval_.get_KS(predictions_pycaret, 'IsFirstDefault', 'Score')